In [1]:
# import the libs.
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import random
import base64

In [2]:
# helper: enc secret with some algs :)

def enc_secret(secret, alg=0):
    to_ret = ''
    if alg == 0:
        kc = get_random_bytes(16)
        iv = get_random_bytes(16)
        aes_obj = AES.new(kc, AES.MODE_CBC, iv)
        enc = aes_obj.encrypt(get_random_bytes(secret))
        enc_base64 = str(base64.b64encode(enc), 'utf8')
        to_ret = [ord(x) for x in enc_base64]
    
    if alg == 1:
        to_ret = [random.randint(0, 256) for i in range(secret)] # random plain-ascii-chars
        
    return to_ret
 
    
# helpers: generate n enc_secrets

def get_n_enc_secrets(n_secrets, len_secret=32, alg=0): # alg = 0 for AES, 1 for random_str
    to_ret = list()
    
    for i in range(n_secrets):
        encoded = enc_secret(len_secret, alg)[:len_secret]
        
        if alg == 0:
            encoded.extend([1, 0])
            to_ret.append(encoded)
            
        if alg == 1:
            encoded.extend([0, 1])
            to_ret.append(encoded)
        
    return to_ret

In [3]:
import pandas as pd

In [4]:
aes_enc = get_n_enc_secrets(50000, alg=0) # generate 50k ciphertexts
print('Done aes')

rand_enc = get_n_enc_secrets(50000, alg=1) # generate 50k rand_str
print('Done rand')

data = list()
data.extend(aes_enc)
data.extend(rand_enc) # create one dataframe with all generated texts

df = pd.DataFrame(data)
df.to_csv('data_for_experiment.csv')

Done aes
Done rand


In [5]:
df = df.sample(frac=1).reset_index(drop=True) # shuffle the dataframe

In [6]:
train = df.sample(frac=0.99,random_state=200) # split in 99k and 1 k, 99k for train
others = df.drop(train.index)

In [7]:
cross = others.sample(frac=0.5,random_state=200) # stplit in 500 for dev set and 500 for test set
test = others.drop(cross.index)

In [8]:
# save the datasets
train.to_csv('train.csv')
cross.to_csv('cross.csv')
test.to_csv('test.csv')